In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import os
import collections
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
######### Valence And Arousal
def SVMPCA(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #Predict Valence SVM and PCA
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    #Predict Arousal SVM and PCA
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
        
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    clf = SVC()
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]

######### Happy And sad ##############
def SVMPCAHS(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #Predict Valence SVM and PCA
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

######### All Emotions##############
def SVMPCACombined(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #Predict Valence SVM and PCA
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]


############## Run Each Method Multiple Times ######################
def Run100(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        svm=SVMPCA(UserName,Components,Window,StartTime,EndTime)
        arousal+=[svm[0]]
        valence+=[svm[1]]
        F1_ScoreArousal+=[svm[2]]
        F1_ScoreValence+=[svm[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];

def RunCombined100(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=SVMPCACombined(UserName,Components,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=SVMPCAHS(UserName,Components,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];
        

    
##################Main#########################33
if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,33):
        print("s"+str(i))
        #Happy and Sad
        if not ("s"+str(i) == "s31"):
            var =RunHS100("s"+str(i))
            print("Happy Sad")
            print(var)
            HSAccurecy+=var[0]
            HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for 31 users "+str(HSAccurecy/31))
    print("F1 happy sad for  31 users "+str(HSF1/31))
    print("Accurecy Combined sad for 32 users"+str(CombinedAccurecy/32))
    print("F1 Combined sad for  32 users "+str(CombinedF1/32))
    print("Accurecy Valence for  32 users "+str(ValenceAccurecy/32))
    print("F1 Valence for 32 users "+str(ValenceF1/32))
    print("Accurecy Arousal for  32 users "+str(ArousalAccurecy/32))
    print("F1 Arousal for  32 users "+str(ArousalF1/32))



s01
Array of average Happy Sad accurecies for user s01
Counter({60.0: 48, 40.0: 31, 20.0: 13, 80.0: 8})
Happy Sad
[50.2, 0.4263968253968254]
Array of average All Emotions accurecies for user s01
Counter({37.5: 36, 25.0: 21, 12.5: 19, 50.0: 17, 62.5: 5, 0.0: 2})
Combined Emotions
[32.75, 0.26176262626262625]
Arousal 58.25
Counter({62.5: 36, 50.0: 32, 75.0: 15, 37.5: 9, 87.5: 5, 25.0: 3})
Valence 37.375
Counter({25.0: 30, 37.5: 28, 50.0: 24, 62.5: 8, 12.5: 7, 75.0: 2, 0.0: 1})
Arousal Valence
[58.25, 37.375, 0.473717698967699, 0.3374112554112554]
s02
Array of average Happy Sad accurecies for user s02
Counter({80.0: 51, 100.0: 37, 60.0: 11, 40.0: 1})
Happy Sad
[84.8, 0.7844523809523808]
Array of average All Emotions accurecies for user s02
Counter({50.0: 36, 37.5: 28, 62.5: 17, 25.0: 10, 12.5: 4, 75.0: 4, 87.5: 1})
Combined Emotions
[46.0, 0.33619520757020765]
Arousal 64.0
Counter({62.5: 38, 75.0: 26, 50.0: 18, 87.5: 9, 37.5: 7, 25.0: 1, 100.0: 1})
Valence 67.625
Counter({75.0: 32, 62.5: 

Array of average Happy Sad accurecies for user s15
Counter({50.0: 48, 75.0: 26, 25.0: 24, 0.0: 2})
Happy Sad
[49.5, 0.4161904761904762]
Array of average All Emotions accurecies for user s15
Counter({37.5: 30, 25.0: 23, 50.0: 20, 12.5: 16, 62.5: 7, 0.0: 3, 75.0: 1})
Combined Emotions
[34.125, 0.2812152777777778]
Arousal 43.75
Counter({37.5: 28, 62.5: 22, 25.0: 21, 50.0: 20, 75.0: 5, 12.5: 4})
Valence 60.5
Counter({62.5: 31, 50.0: 25, 75.0: 19, 37.5: 14, 87.5: 9, 100.0: 1, 25.0: 1})
Arousal Valence
[43.75, 60.5, 0.3903419358419359, 0.5916903929403928]
s16
Array of average Happy Sad accurecies for user s16
Counter({80.0: 29, 60.0: 28, 100.0: 21, 40.0: 16, 20.0: 6})
Happy Sad
[68.6, 0.639563492063492]
Array of average All Emotions accurecies for user s16
Counter({25.0: 31, 37.5: 28, 50.0: 14, 62.5: 13, 12.5: 7, 0.0: 5, 75.0: 2})
Combined Emotions
[35.75, 0.2974910714285714]
Arousal 62.625
Counter({62.5: 33, 75.0: 21, 50.0: 17, 37.5: 12, 87.5: 11, 100.0: 3, 25.0: 2, 12.5: 1})
Valence 59.5
C

Array of average Happy Sad accurecies for user s29
Counter({83.33333333333334: 33, 66.66666666666666: 31, 50.0: 18, 100.0: 13, 33.33333333333333: 5})
Happy Sad
[71.83333333333334, 0.6526577681577682]
Array of average All Emotions accurecies for user s29
Counter({50.0: 32, 62.5: 28, 75.0: 16, 37.5: 15, 25.0: 5, 87.5: 3, 12.5: 1})
Combined Emotions
[55.125, 0.4350286657786658]
Arousal 70.25
Counter({75.0: 32, 62.5: 27, 87.5: 19, 50.0: 12, 100.0: 5, 37.5: 4, 25.0: 1})
Valence 60.125
Counter({50.0: 28, 62.5: 28, 75.0: 21, 37.5: 11, 87.5: 8, 25.0: 3, 100.0: 1})
Arousal Valence
[70.25, 60.125, 0.6146451048951049, 0.5789662282162282]
s30
Array of average Happy Sad accurecies for user s30
Counter({71.42857142857143: 38, 57.14285714285714: 29, 85.71428571428571: 20, 42.857142857142854: 11, 100.0: 1, 28.57142857142857: 1})
Happy Sad
[66.85714285714286, 0.5482069042069042]
Array of average All Emotions accurecies for user s30
Counter({50.0: 35, 62.5: 25, 75.0: 18, 37.5: 16, 25.0: 3, 87.5: 2, 100.